In [4]:
console.log("Hello, world!")

Hello, world!


This tutorial goes through a couple of examples that show the evolution of E-ID solutions.

We will go through the following examples 

1. Typical message signing without privacy
2. Adding Selective Disclosure
3. Adding Zero-Knowledge proofs
4. Adding Unlinkability

In [5]:
// We start by creating a typical E-ID object that we will use through out this tutorial

const EID_DATA = {
    name: "Ahmed",
    age: "22",
    profession: "RSE"
}

## A typical message signature in today's world

In [7]:
// Sender 
import * as crypto from 'crypto';

// Creating Key Pair
const { publicKey, privateKey } = crypto.generateKeyPairSync('rsa', {
    modulusLength: 4096,
    publicKeyEncoding: {
      type: 'spki',   

      format: 'pem'
    },
    privateKeyEncoding: {
      type: 'pkcs8',
      format: 'pem',
    }
  });

const message = JSON.stringify(EID_DATA)

// sign the secret data 
const signer = crypto.createSign('SHA256');
signer.update(message);
const signature = signer.sign(privateKey, 'base64')

Holder maintains a single signature at all times.  
Everytime it is presented, it's the same!

In [8]:
// Receiver - receives message, signature, and publicKey

let verifier = crypto.createVerify('SHA256');
verifier.update(message)
console.log(verifier.verify(publicKey, signature, 'base64'))

// try another false message
verifier = crypto.createVerify('SHA256');
verifier.update("Another random message!")
console.log(verifier.verify(publicKey, signature, 'base64'))

true
false


Right now, we need to send all of the EID_DATA content to be able to verify it. 

What if we want to send just the name, and verify that? 

RSA signatures are holistic; you either verify the entire signed message or nothing at all.

To do this, an alternative method should we used. In the following sections we will use a modern crypto scheme called BBS+

In [9]:
import { BBS } from './src/bbs';

In [10]:
const bbs = new BBS();

// generate key pair
const privateKey = bbs.KeyGen(crypto.randomBytes(32));
const publicKey = bbs.SkToPk(privateKey);

In [11]:

// preparing the data
const messagesToSign = Object.values(EID_DATA)

// we hash the messages (One-way process)
let hashedMessages = messagesToSign.map(message => bbs.MapMessageToScalarAsHash(Buffer.from(message)));

// signing the thing
const generators = await bbs.create_generators(messagesToSign.length);
const signature = bbs.Sign(privateKey, publicKey, Buffer.from("HEADER", "utf-8"), hashedMessages, generators)

// let's verify these hashes before we send things out 
bbs.Verify(publicKey, signature, Buffer.from("HEADER", "utf-8"), hashedMessages, generators);

In [42]:
signature

Uint8Array(80) [
  184,  65, 164, 247, 119, 206,  16, 151, 102,  82, 197, 122,
    4, 169, 214,  31,  73,  25, 231,  90, 202,  92,  38,  94,
  160, 243, 196, 171,  31, 164,  19,  62,  12, 246, 167, 108,
  158, 252,  61,   8,  74, 253,  66, 245, 202, 228,  51, 140,
   89,  47,  21, 101, 186,  54, 219, 166,  59, 178, 217, 180,
  131, 212, 158, 205,  59, 228, 141,  76, 165,  66,  30, 132,
  156, 195,  38, 188, 229, 228, 231,  83
]


Both the EID_DATA and signature are transmitted to the EID holder.

Then, we assume the holder/ owner of the EID wants to only share name, and profession with a certain entity. 

## Holder

In [14]:

// holder determines that they want to disclose only the first and last messages (name, profession)
const IndexesOfFieldsToDisclose = [0,2]

// const HolderGenerators = await bbs.create_generators(messagesToSign.length);

// holder generates a proof
const proof = bbs.ProofGen(
    publicKey, signature, 
    Buffer.from("HEADER", "utf-8"), Buffer.from("PRESENTATION HEADER", "utf-8"),
    hashedMessages, generators, IndexesOfFieldsToDisclose
)
const MessagesToDisclose = messagesToSign.filter((val, i) => {return IndexesOfFieldsToDisclose.includes(i)})

MessagesToDisclose

[ 'Ahmed', 'RSE' ]


In [20]:
// Verifier - received publicKey, proof, disclosed data, disclosed data indexes, all hashed messages

const verifierGenerators = await bbs.create_generators(messagesToSign.length);
let verifierHashedMessages = MessagesToDisclose.map(message => bbs.MapMessageToScalarAsHash(Buffer.from(message)));

bbs.ProofVerify(
    publicKey, proof,
    Buffer.from("HEADER", "utf-8"), Buffer.from("PRESENTATION HEADER", "utf-8"),
    verifierHashedMessages, verifierGenerators, IndexesOfFieldsToDisclose
);

MessagesToDisclose

[ 'Ahmed', 'RSE' ]


In [18]:
// Testing with fake data
const fakeMessages = ['Ahmed', 'CEO']

const verifierGenerators = await bbs.create_generators(messagesToSign.length);

// we use fakeMessages instead of MessagesToDisclose here
let verifierHashedMessages = fakeMessages.map(message => bbs.MapMessageToScalarAsHash(Buffer.from(message)));

bbs.ProofVerify(
    publicKey, proof,
    Buffer.from("HEADER", "utf-8"), Buffer.from("PRESENTATION HEADER", "utf-8"),
    verifierHashedMessages, verifierGenerators, IndexesOfFieldsToDisclose
);

MessagesToDisclose

Invalid proof (cv)


In [37]:
// Unlinkability 

const customGenerators = await bbs.create_generators(messagesToSign.length);

const proof1 = bbs.ProofGen(
    publicKey, signature, 
    Buffer.from("HEADER", "utf-8"), Buffer.from("PRESENTATION HEADER", "utf-8"),
    hashedMessages, customGenerators, IndexesOfFieldsToDisclose
)

const proof2 = bbs.ProofGen(
    publicKey, signature, 
    Buffer.from("HEADER", "utf-8"), Buffer.from("PRESENTATION HEADER", "utf-8"),
    hashedMessages, customGenerators, IndexesOfFieldsToDisclose
)

proof1 == proof2

false


In [40]:
new Buffer(proof2).toString("hex")

9616cfc41adb753c7c5a497d96a3d224c464eb7b8e34c2b24f567a2894dc5fc0bf8df7f5a96531578baf7329006f037eb2df3e8fede8284a06be705525215298aa1e67b3e1339dd497cf422c14477bb29d2d9b1a1f0f448698c1d8cbdfb799db8903c9758342160107dbba542b1b08e2268e1e35a698968ddc88cfe65f7a2049a6b08b2646b64ad4c922974aa91ad76725dac7ebc3174cb39f9f79a2ca5d87b2eee78db77874ea72f4811ecc86a7f6d04d6c51700a8939283893c6259e0c6c5ea8aad7895f2ff5420bae44926fe539470515dbc9e0a8a6a79ca7d6c670d038f128d8a42cc09117431ec8c4dd7f28459529aeb4b3906845616aad0e85b0f8142a7630d85cbd81a76ed44b68a00532a06c1bb3d898852053d698c0c69dd7e3495f8543c462a05ca5a2eebe14fdc75ce03d


In [41]:
new Buffer(proof1).toString("hex")

932c1b994f6408ce1b64581225928e72208f5edf68111c1c5e146e3fd4821abcbbb641795846325728b178ffd4f604d889f8650816efd4cc09af91aa2138345867dea4bb634a28ceed9019892b4ac55e79f4509413d8a2f63c83e9e2e7fa7592b144ed1efa50df08086c28fe6248c2dff505677bb9a962e690de543c588cb4dda0335d73051762df2c854dc2e2277c371375fefd7453eafc6c7a58381e05a21d0b2b81bcbcc2d447d977a6a7caccd92c2855c834f7784587171dd994450a6eff0b9259a866bc676ecfdf22d8427c7c3c2b8af9d01d6f01caf84624c8fabf626c752cf5155cff04da1855732f7de794051b7b337981c3259141306d2d459fd381ed8cf859e78600c36d33510a85e657246e9bd4a9299bd6e18247632e7f2a137e6f8e1d6c9c3211493abe52e8a347bc98
